In [1]:
#!/usr/bin/python3

from SchemaLOD import *
from config import default_crosswalks_location, crosswalks_location

schema = Schema()

In [2]:
defaultvalue = schema.default_schema(default_crosswalks_location)
defaultvalue

{'subject': 'Arts and Humanities',
 'authorName': 'Slava',
 'authorAffiliation': 'DANS',
 'datasetContactEmail': 'vty@dans.knaw.nl',
 'datasetContactName': 'Slava'}

In [3]:
crosswalks = schema.crosswalks(crosswalks_location)
crosswalks

{'investigator': 'authorName',
 'organization': 'affiliation',
 'taxon': 'dsDescription',
 'category': 'keyword'}

In [4]:
schemaURL = 'https://raw.githubusercontent.com/IQSS/dataverse/develop/scripts/api/data/metadatablocks/citation.tsv'
schemapd = schema.load_metadata_schema(schemaURL, 'citation')
schema.to_graph('citation', filename='citation')
schema.metadataframe

b'Skipping line 72: expected 17 fields, saw 18\nSkipping line 85: expected 17 fields, saw 22\n'


Index(['#datasetField', 'name', 'title', 'description', 'watermark',
       ' fieldType', 'displayOrder', 'displayFormat', 'advancedSearchField',
       'allowControlledVocabulary', 'allowmultiples', 'facetable',
       'displayoncreate', 'required', 'parent', 'metadatablock_id', 'termURI'],
      dtype='object')


,#datasetField,name,title,description,watermark,fieldType,displayOrder,displayFormat,advancedSearchField,allowControlledVocabulary,allowmultiples,facetable,displayoncreate,required,parent,metadatablock_id,termURI
0,,title,Title,Full title by which the Dataset is known.,Enter title...,text,0.0,,True,False,False,False,True,True,,citation,http://purl.org/dc/terms/title
1,,subtitle,Subtitle,A secondary title used to amplify or state cer...,,text,1.0,,False,False,False,False,False,False,,citation,
2,,alternativeTitle,Alternative Title,A title by which the work is commonly referred...,,text,2.0,,False,False,False,False,False,False,,citation,http://purl.org/dc/terms/alternative
3,,alternativeURL,Alternative URL,"A URL where the dataset can be viewed, such as...","Enter full URL, starting with http://",url,3.0,"<a href=""#VALUE"" target=""_blank"">#VALUE</a>",False,False,False,False,False,False,,citation,https://schema.org/distribution
4,,otherId,Other ID,Another unique identifier that identifies this...,,none,4.0,:,False,False,True,False,False,False,,citation,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,,relatedDatasets,Related Datasets,"Any Datasets that are related to this Dataset,...",,textbox,72.0,,False,False,True,False,False,False,,citation,http://purl.org/dc/terms/relation
69,,otherReferences,Other References,Any references that would serve as background ...,,text,73.0,,False,False,True,False,False,False,,citation,http://purl.org/dc/terms/references
70,,originOfSources,Origin of Sources,"For historical materials, information about th...",,textbox,75.0,,False,False,False,False,False,False,,citation,
71,,characteristicOfSources,Characteristic of Sources Noted,Assessment of characteristics and source mater...,,textbox,76.0,,False,False,False,False,False,False,,citation,


In [5]:
defaultmetadata = {}
for i in schema.default.index:
    fieldname = str(schema.default.loc[i]['defaultfield'])
    print(fieldname)
    cfields = schema.Hierarchy(fieldname)
    print(cfields)
    if not cfields['fields']:
        # field without children
        triples = schema.Relations(fieldname, NESTED=True, relation='#exactMatch') 
        if triples:
            defaultmetadata[schema.get_object(triples[0])] = schema.default.loc[i]['value']
    else:
        metadatablock = {}
        for extrafield in cfields['fields']:
            triples = schema.Relations(extrafield, NESTED=True, relation='#altLabel') 
            print("\t %s => %s " % (extrafield, schema.get_object(triples[0])))
            if schema.get_object(triples[0]) in defaultvalue:
                vocnameS = schema.get_subject(triples[0])
                vocname = schema.vocURI(vocnameS)
                metadatablock[vocname] = defaultvalue[schema.get_object(triples[0])]
        defaultmetadata[schema.rootURI(schema.termURI(cfields['root']))] = metadatablock
        
        #print(cfields['root'])
        #print(schema.termURI(cfields['root']))

subject
{'fields': []}
authorName
{'root': 'https://dataverse.org/schema/citation/author', 'fields': ['https://dataverse.org/schema/citation/authorName', 'https://dataverse.org/schema/citation/authorAffiliation', 'https://dataverse.org/schema/citation/authorIdentifier']}
	 https://dataverse.org/schema/citation/authorName => authorName 
	 https://dataverse.org/schema/citation/authorAffiliation => authorAffiliation 
	 https://dataverse.org/schema/citation/authorIdentifier => authorIdentifier 
authorAffiliation
{'root': 'https://dataverse.org/schema/citation/author', 'fields': ['https://dataverse.org/schema/citation/authorName', 'https://dataverse.org/schema/citation/authorAffiliation', 'https://dataverse.org/schema/citation/authorIdentifier']}
	 https://dataverse.org/schema/citation/authorName => authorName 
	 https://dataverse.org/schema/citation/authorAffiliation => authorAffiliation 
	 https://dataverse.org/schema/citation/authorIdentifier => authorIdentifier 
datasetContactEmail
{'ro

In [6]:
print(json.dumps(defaultmetadata, indent=4))

{
    "http://purl.org/dc/terms/subject": "Arts and Humanities",
    "http://purl.org/dc/terms/creator": {
        "https://dataverse.org/schema/citation/author#Name": "Slava",
        "https://dataverse.org/schema/citation/author#Affiliation": "DANS"
    },
    "https://dataverse.org/schema/citation/Contact": {
        "https://dataverse.org/schema/citation/datasetContact#Name": "Slava",
        "https://dataverse.org/schema/citation/datasetContact#E-mail": "vty@dans.knaw.nl"
    }
}


In [7]:
fields = schema.Overview('schema_required', 'True')
fields

{'title': {'uri': 'https://dataverse.org/schema/citation/title',
  'loc': rdflib.term.BNode('N8cb66a5b86824ad49fb92e59c7c7f37c')},
 'author': {'uri': 'https://dataverse.org/schema/citation/author',
  'loc': rdflib.term.BNode('N93a85093bf834bda812e0aa0d1356f8a')},
 'authorName': {'uri': 'https://dataverse.org/schema/citation/authorName',
  'loc': rdflib.term.BNode('Nbd0500cca13c46ea8ef3ad66c2b740fe')},
 'datasetContact': {'uri': 'https://dataverse.org/schema/citation/datasetContact',
  'loc': rdflib.term.BNode('Ne99b6d9255bd4314a57b9d36b66013b8')},
 'datasetContactEmail': {'uri': 'https://dataverse.org/schema/citation/datasetContactEmail',
  'loc': rdflib.term.BNode('N5e23191e851d411896ccdae0354f56f7')},
 'dsDescription': {'uri': 'https://dataverse.org/schema/citation/dsDescription',
  'loc': rdflib.term.BNode('Ncac4bfe953374434b95df2b6c1be076d')},
 'dsDescriptionValue': {'uri': 'https://dataverse.org/schema/citation/dsDescriptionValue',
  'loc': rdflib.term.BNode('Nae8e854aff404690bb4f

In [8]:
csvfile = 'https://raw.githubusercontent.com/Dans-labs/common-migrations/master/core/resources/examples/example.csv'
dataset = schema.loadfile(csvfile)
dataset

,title,identifier,sid,state,permission,organization,lat,lng,firstYear,lastYear,taxon,objectType,elementType,investigator,language,category,UserLastname,userInitials,userEmail,userId
0,"CH, Co. Amsterdam",Klc House,dccd:4742,PUBLISHED,values,Quebec Univ,12.456,-9.098,8768,9887,Curques,House,NaN,Lian McDonald,en,built in plug,Blue,K. M.,k.blue@example.uk,King


In [9]:
newfields = []
for field in dataset.columns:
    if field in crosswalks:
        newfields.append(crosswalks[field])
    else:
        newfields.append(field)
dataset.columns = newfields
dataset

,title,identifier,sid,state,permission,affiliation,lat,lng,firstYear,lastYear,dsDescription,objectType,elementType,authorName,language,keyword,UserLastname,userInitials,userEmail,userId
0,"CH, Co. Amsterdam",Klc House,dccd:4742,PUBLISHED,values,Quebec Univ,12.456,-9.098,8768,9887,Curques,House,NaN,Lian McDonald,en,built in plug,Blue,K. M.,k.blue@example.uk,King


In [10]:
metadata = defaultmetadata
for i in dataset.index:
    data = dataset.loc[i]
    for col in range(0, len(data)):
        thisfield = dataset.columns[col]
        thisvalue = dataset.loc[i][thisfield]
        print("Metadata [%s] %s %s => %s"% (col, dataset.columns[col], data[col], thisfield)) 
        nested = schema.Hierarchy(thisfield)
        if 'root' in nested:
            print(nested)
            metadatablock = {}
            valuefield = "%sValue" % thisfield
            for field in nested['fields']:                
                #if field in newfields:
                print("\t\t %s %s" % (field, schema.vocURI(field)))
                if schema.termURI(field) == schema.termURI(thisfield) or 'Value' in field: 
                    #schema.termURI(field) == schema.termURI(valuefield):
                    metadatablock[schema.vocURI(field)] = thisvalue
            print(metadatablock)
            metadata[schema.rootURI(nested['root'])] = metadatablock
        else:
            termURI = schema.termURI(thisfield)
            if termURI:
                print("\t %s" % termURI)
                metadata[termURI] = thisvalue

Metadata [0] title CH, Co. Amsterdam => title
	 http://purl.org/dc/terms/title
Metadata [1] identifier Klc House => identifier
Metadata [2] sid dccd:4742 => sid
Metadata [3] state PUBLISHED => state
Metadata [4] permission values => permission
Metadata [5] affiliation Quebec Univ => affiliation
Metadata [6] lat 12.456 => lat
Metadata [7] lng -9.097999999999999 => lng
Metadata [8] firstYear 8768 => firstYear
Metadata [9] lastYear 9887 => lastYear
Metadata [10] dsDescription Curques => dsDescription
{'root': 'https://dataverse.org/schema/citation/dsDescription', 'fields': ['https://dataverse.org/schema/citation/dsDescriptionValue', 'https://dataverse.org/schema/citation/dsDescriptionDate']}
		 https://dataverse.org/schema/citation/dsDescriptionValue https://dataverse.org/schema/citation/dsDescription#Text
		 https://dataverse.org/schema/citation/dsDescriptionDate https://dataverse.org/schema/citation/dsDescription#Date
{'https://dataverse.org/schema/citation/dsDescription#Text': 'Curques

In [11]:
def rootURI(fieldname):
    triplesubj = schema.Relations(fieldname, NESTED=True, relation='#prefLabel')
    if 'http' in triplesubj[0]['o']:
        return triplesubj[0]['o']
    return 

    #return ("%s#%s" % (triples[0]['o'], triplesubj[0]['o']))
def vociURI(fieldname):
    triples = schema.Relations(fieldname, NESTED=True, relation='#broader')
    triplesubj = schema.Relations(fieldname, NESTED=True, relation='#prefLabel')
    return ("%s#%s" % (triples[0]['o'], triplesubj[0]['o']))

fieldname= 'https://dataverse.org/schema/citation/dsDescriptionValue'
fieldname = 'https://dataverse.org/schema/citation/datasetContactEmail'
fieldname = 'https://dataverse.org/schema/citation/datasetContact'
#fieldname = 'https://dataverse.org/schema/citation/dsDescription'
rootURI(fieldname)

'https://dataverse.org/schema/citation/Contact'

In [12]:
print(json.dumps(metadata, indent=4))

{
    "http://purl.org/dc/terms/subject": "Arts and Humanities",
    "http://purl.org/dc/terms/creator": {
        "https://dataverse.org/schema/citation/author#Name": "Slava",
        "https://dataverse.org/schema/citation/author#Affiliation": "DANS"
    },
    "https://dataverse.org/schema/citation/Contact": {
        "https://dataverse.org/schema/citation/datasetContact#Name": "Slava",
        "https://dataverse.org/schema/citation/datasetContact#E-mail": "vty@dans.knaw.nl"
    },
    "http://purl.org/dc/terms/title": "CH, Co. Amsterdam",
    "https://dataverse.org/schema/citation/Description": {
        "https://dataverse.org/schema/citation/dsDescription#Text": "Curques"
    },
    "https://dataverse.org/schema/citation/Author": {
        "https://dataverse.org/schema/citation/author#Name": "Lian McDonald"
    },
    "https://dataverse.org/schema/citation/Keyword": {
        "https://dataverse.org/schema/citation/keyword#Term": "built in plug"
    }
}
